# Logistic Regression

Refs:
Apply feature selection is a mandatory stuff for logistic regression, via PCA or Variace Threshold and others, worths the reading: 
https://medium.com/@data.science.enthusiast/feature-selection-techniques-forward-backward-wrapper-selection-9587f3c70cfa

In [10]:
import pandas as pd
import numpy as np
from spaceship_titanic import feature_enginnering as fe

In [7]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [11]:

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = fe.cabin_inputer(df)
    df = fe.feature_inputer(df)
    df = fe.vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = fe.outliers_to_log(df)
    df = fe.dtype_memory_reducer(df)
    return df
def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [12]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [13]:
#df_train = apply_interactions(df_train, 'Transported')

In [14]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

## Evaluation

In [16]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)

In [17]:
scores = cross_validate(
    model,
    X_train,
    y_train,
    cv=KFold(5, shuffle=True, random_state=42),
    scoring = "accuracy",
    return_train_score=True
)

In [18]:
scores

{'fit_time': array([0.04399657, 0.02309108, 0.03126454, 0.03124928, 0.0312705 ]),
 'score_time': array([0.00199842, 0.        , 0.        , 0.        , 0.        ]),
 'test_score': array([0.77055779, 0.76308223, 0.76423232, 0.77272727, 0.78596087]),
 'train_score': array([0.77322404, 0.77408686, 0.7768191 , 0.77325665, 0.77009346])}

In [19]:
results = {
    'train_score_avg':np.mean(scores['train_score']),
    'train_score_std':np.std(scores['train_score']),
    'test_score_avg':np.mean(scores['test_score']),
    'test_score_std':np.std(scores['test_score']),
}
results

{'train_score_avg': 0.7734960209920093,
 'train_score_std': 0.002149447635064266,
 'test_score_avg': 0.7713120975442548,
 'test_score_std': 0.008185503690592694}

# Submission

In [20]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [52]:
y_test = model.predict(X_test)

In [53]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_rnforest_2022-11-02.csv')